In [1]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost"])

0

In [2]:
import sys
print(sys.executable)

c:\Users\jiamv\miniconda3\python.exe


In [3]:
import pandas as pd 
import xgboost 
df = pd.read_excel(r'C:\Users\jiamv\Downloads\Low_Conc_Experiments.xlsx')

x = df.iloc[:, [1, 9, 10, 11]]
y = df.iloc[:, list(range(1, 3))]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from xgboost import XGBRegressor

xg = XGBRegressor(n_estimators=100, random_state=42)
xg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [4]:
x.head()

Time in minutes    SPS30_01    SPS30_03    SPS30_04
0                1  155.182222  276.164444  139.057500
1                2  151.732500  271.147500  131.615556
2                3  160.726667  262.652222  142.931111
3                4  239.026667  259.520000  241.676667
4                5  299.777778  256.345556  271.020000

In [6]:
y_pred = xg.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)
y_pred_df['Time'] = df['Time in minutes']
y_pred_df 

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared Score:", r2)
print("Weights:", xg.feature_importances_)

Mean Absolute Error: 2.0506153106689453
Mean Squared Error: 8.159405708312988
R-squared Score: 0.9979250431060791
Weights: [0.88609356 0.05453994 0.01316341 0.04620313]


In [15]:
# Print XGBoost parameters
print(xg.get_params())

# Access learning rate
print(xg.learning_rate)

{'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'feature_weights': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
None


In [7]:
# Use full dataset or a large subset
X_distill = x.copy()

# XGBoost predictions (teacher outputs)
y_distill = xg.predict(X_distill)


In [8]:
y_distill.shape

(171, 2)

In [9]:
# Split outputs
y1_distill = y_distill[:, 0]
y2_distill = y_distill[:, 1]

In [12]:
from sklearn.ensemble import RandomForestRegressor

student_3 = RandomForestRegressor(
    n_estimators=8,
    max_depth=3,
    random_state=42
)

student_4 = RandomForestRegressor(
    n_estimators=8,
    max_depth=3,
    random_state=42
)

student_3.fit(X_distill, y1_distill)
student_4.fit(X_distill, y2_distill)

RandomForestRegressor(max_depth=3, n_estimators=8, random_state=42)

In [13]:
y1_student_pred = student_3.predict(X_test)
y2_student_pred = student_4.predict(X_test)

from sklearn.metrics import r2_score

print("Output 1 R2:", r2_score(y_test.iloc[:,0], y1_student_pred))
print("Output 2 R2:", r2_score(y_test.iloc[:,1], y2_student_pred))
print("Weights:", student_3.feature_importances_)
print("Weights:", student_4.feature_importances_)

Output 1 R2: 0.993720019162189
Output 2 R2: 0.9905926450190724
Weights: [0.35221128 0.12452092 0.28348875 0.23977905]
Weights: [0.35366711 0.15626232 0.24784775 0.24222282]


In [16]:
# Print details of first tree
print("Tree details:", student_3.estimators_[0])

# Get tree depth
print("Tree depth:", student_3.estimators_[0].get_depth())

# Print details of first tree
print("Tree details:", student_4.estimators_[0])

# Get tree depth
print("Tree depth:", student_4.estimators_[0].get_depth())

Tree details: DecisionTreeRegressor(max_depth=3, max_features=1.0, random_state=1608637542)
Tree depth: 3
Tree details: DecisionTreeRegressor(max_depth=3, max_features=1.0, random_state=1608637542)
Tree depth: 3


In [14]:
import micromlgen
from micromlgen import port

with open("model_output3.h", "w") as f:
    f.write(port(student_3))

with open("model_output4.h", "w") as f:
    f.write(port(student_4))